In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd gdrive

/content/gdrive


In [ ]:
cd MyDrive

/content/gdrive/MyDrive


In [ ]:
cd Colab datasets

/content/gdrive/MyDrive/Colab datasets


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as mplot

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

import math

In [ ]:
Data = pd.read_csv("processed.csv")
Data.head()

,Category,CounterSpeech,commentText,date,hasReplies,id,likes,numberOfReplies,timestamp,user,Community,replies
0,Default,False,last brave white women brain,5 years ago,0.0,Ugw-_mfnTo9SUniGGTN4AaABAg,0,0.0,2012-12-05 09:13:58.535,Ik hou van surfen,jews,[]
1,Default,False,amen pastor,2 years ago,0.0,UgiMOgO3u4akkngCoAEC,0,0.0,2015-12-05 04:47:36.420,julius smith,lgbt,[]
2,"2,8",True,kind god hates people curses kind people belie...,4 years ago,0.0,UgzemCMbg-ri1ri7sCt4AaABAg,0,0.0,2013-12-05 09:16:05.293,gary wersley,jews,[]
3,8,True,question comment racist jews smartest group pe...,3 years ago,0.0,UghFhzrURF1svHgCoAEC,0,0.0,2015-01-05 16:27:25.589,Les Ordway,jews,[]
4,8,True,israel fail nothing new israel rebelious natio...,5 years ago,0.0,Ugzr50sQdH0Ictuex214AaABAg,0,0.0,2012-12-05 09:17:14.596,HamalachMichael,jews,[]


In [ ]:
Data.dropna(axis=0, inplace=True)

## Logistic Regression + Bag of Words Vector

In [ ]:
#import mglearn
train_fraction = 0.7
train_index = math.floor(Data.shape[0]*train_fraction)

train_data = Data.loc[:train_index, 'commentText'].values
train_label = Data.loc[:train_index, 'CounterSpeech'].values
test_data = Data.loc[train_index:, 'commentText'].values
test_label = Data.loc[train_index:, 'CounterSpeech'].values

YTrain = train_label
YTest = test_label

In [ ]:
print(train_data)

['last brave white women brain' 'amen pastor'
 'kind god hates people curses kind people believe god roots islamicfundamentalism happen muslims xterminated jews find others god hates persecute strange religion seems religion hatred muslims seem hate faithwith religious beliefs never peace among mankind'
 ...
 'hugh ridiculous nobodys making millions dollars invested propry stead snorting families would st fot life got money 💰 thy talent point jews nothing money 💰 buy blue chip tocks bnot ferraris'
 'suit wearingiam pretty sure material came frm jews garment factory'
 'course hes obvioudly major shareholder snd ceo theres exxon shell imb chevron walmart etc jewish cpsnies according uour criteria definitely real monrybis eith thr olf established wasps sre number successful jews catholiv vhurch lsrgest land osner vountry day jews run country money idnt true sndcbelirve tluou know problem rhevmondy rsrned homestly fsct modtbsre liberals support people like obama seems like foreign agent kn

In [ ]:
print(YTrain)

[False False  True ...  True  True False]


In [ ]:
vect = CountVectorizer(min_df=2, ngram_range=(1, 3))
XTrain = vect.fit(train_data).transform(train_data)
XTest = vect.transform(test_data)

In [ ]:
print("Vocabulary size: {}".format(len(vect.vocabulary_)))
print("XTrain : \n{}".format(repr(XTrain)))
print("XTest : \n{}".format(repr(XTest)))
print("Features : {}".format(len(vect.get_feature_names())))

Vocabulary size: 24677
XTrain : 
<9661x24677 sparse matrix of type '<class 'numpy.int64'>'
	with 187030 stored elements in Compressed Sparse Row format>
XTest : 
<4209x24677 sparse matrix of type '<class 'numpy.int64'>'
	with 61858 stored elements in Compressed Sparse Row format>
Features : 24677


In [ ]:
parameter_grid = {'C': [0.1, 0.2, 0.3, 0.4, 0.5]}
grid = GridSearchCV(LogisticRegression(), parameter_grid, cv = 5)
grid.fit(XTrain, YTrain)

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 0.2, 0.3, 0.4, 0.5]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
print("Best cross-validation score: {}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)
print("Best estimator: ", grid.best_estimator_)

Best cross-validation score: 0.6974435515225906
Best parameters:  {'C': 0.3}
Best estimator:  LogisticRegression(C=0.3, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [ ]:
lr = grid.best_estimator_
lr.fit(XTrain, YTrain)

LogisticRegression(C=0.3, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
lr.predict(XTest)
print("Score: {}".format(lr.score(XTest, YTest)))

Score: 0.62984081729627


In [ ]:
ypred = lr.predict(XTest)
print(classification_report(YTest,ypred))

              precision    recall  f1-score   support

       False       0.48      0.83      0.61      1478
        True       0.85      0.52      0.65      2731

    accuracy                           0.63      4209
   macro avg       0.67      0.67      0.63      4209
weighted avg       0.72      0.63      0.63      4209



## Random Forest + Bag of Words Vector

In [ ]:
train_data_features=XTrain.toarray()
forest = RandomForestClassifier(n_estimators = 150) 
forest = forest.fit( train_data_features, YTrain)

In [ ]:
test_data_features=XTest.toarray()
result = forest.predict(test_data_features)
print(result)

[False False False ... False False  True]


In [ ]:
print(classification_report(YTest,result))

              precision    recall  f1-score   support

       False       0.48      0.78      0.59      1478
        True       0.82      0.54      0.65      2731

    accuracy                           0.63      4209
   macro avg       0.65      0.66      0.62      4209
weighted avg       0.70      0.63      0.63      4209



## Multinomial Naive Bayes + Bag of Words Vector

In [ ]:

naive = MultinomialNB()
classifier_naive = naive.fit(XTrain,YTrain)
predict_naive = classifier_naive.predict(XTest)

print(predict_naive)

[False False False ... False  True  True]


In [ ]:
print(classification_report(YTest,predict_naive))

              precision    recall  f1-score   support

       False       0.52      0.73      0.61      1478
        True       0.82      0.63      0.71      2731

    accuracy                           0.67      4209
   macro avg       0.67      0.68      0.66      4209
weighted avg       0.71      0.67      0.68      4209



## Support Vector Machine + Bag of Words Vector

In [ ]:
svm = LinearSVC()
classifier_SVM = svm.fit(XTrain, YTrain)
predict_SVM = svm.predict(XTest)
print(predict_SVM)
#print(classification_report(YTest, predict_SVM ))
logreg = LogisticRegression(dual=False)

[False False False ... False  True  True]


In [ ]:
print(classification_report(YTest,predict_SVM))

              precision    recall  f1-score   support

       False       0.47      0.74      0.58      1478
        True       0.80      0.55      0.65      2731

    accuracy                           0.62      4209
   macro avg       0.64      0.65      0.62      4209
weighted avg       0.69      0.62      0.63      4209



## Decision Tree + Bag of Words Vector

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(XTrain, YTrain)
tree_pred = dt.predict(XTest)
print(tree_pred)

[False False  True ... False False  True]


In [ ]:
print(classification_report(YTest, tree_pred))

              precision    recall  f1-score   support

       False       0.46      0.71      0.55      1478
        True       0.77      0.54      0.64      2731

    accuracy                           0.60      4209
   macro avg       0.62      0.63      0.60      4209
weighted avg       0.66      0.60      0.61      4209



## Logistic Regression + TF-IDF

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1, 3), max_df=0.9, min_df=5, sublinear_tf=True)
XTrain = tfidf.fit(train_data).transform(train_data)
XTest = tfidf.transform(test_data)

In [ ]:
print("Vocabulary size: {}".format(len(tfidf.vocabulary_)))
print("XTrain : \n{}".format(repr(XTrain)))
print("XTest : \n{}".format(repr(XTest)))
print("Features : {}".format(len(tfidf.get_feature_names())))

Vocabulary size: 5733
XTrain : 
<9661x5733 sparse matrix of type '<class 'numpy.float64'>'
	with 141418 stored elements in Compressed Sparse Row format>
XTest : 
<4209x5733 sparse matrix of type '<class 'numpy.float64'>'
	with 53161 stored elements in Compressed Sparse Row format>
Features : 5733


In [ ]:
parameter_grid = {'C': [7.9,8,8.25]}
grid = GridSearchCV(LogisticRegression(max_iter = 300), parameter_grid, cv = 5)
grid.fit(XTrain, YTrain)
print("Best cross-validation score: {}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)
print("Best estimator: ", grid.best_estimator_)

Best cross-validation score: 0.6962019045905323
Best parameters:  {'C': 7.9}
Best estimator:  LogisticRegression(C=7.9, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=300,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [ ]:
lr = grid.best_estimator_
lr.fit(XTrain, YTrain)

LogisticRegression(C=7.9, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=300,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
lr.predict(XTest)
print("Score: {}".format(lr.score(XTest, YTest)))

Score: 0.6391066761701116


In [ ]:
ypred = lr.predict(XTest)
print(classification_report(YTest, ypred))

              precision    recall  f1-score   support

       False       0.49      0.75      0.59      1478
        True       0.81      0.58      0.68      2731

    accuracy                           0.64      4209
   macro avg       0.65      0.66      0.63      4209
weighted avg       0.70      0.64      0.65      4209



## Multinomial Naive Bayes + TF-IDF

In [ ]:
naive = MultinomialNB()
classifier_naive = naive.fit(XTrain,YTrain)
predict_naive = classifier_naive.predict(XTest)

print(predict_naive)

[False False False ... False  True  True]


In [ ]:
print(classification_report(YTest, predict_naive))

              precision    recall  f1-score   support

       False       0.49      0.84      0.62      1478
        True       0.86      0.52      0.65      2731

    accuracy                           0.63      4209
   macro avg       0.67      0.68      0.63      4209
weighted avg       0.73      0.63      0.64      4209



## Support Vector Machine + TF-IDF

In [ ]:
svm = LinearSVC()
classifier_SVM = svm.fit(XTrain, YTrain)
predict_SVM = svm.predict(XTest)
print(predict_SVM)
logreg = LogisticRegression(dual=False)

[False False False ... False  True  True]


In [ ]:
print(classification_report(YTest, predict_SVM))

              precision    recall  f1-score   support

       False       0.49      0.74      0.59      1478
        True       0.80      0.58      0.68      2731

    accuracy                           0.64      4209
   macro avg       0.65      0.66      0.63      4209
weighted avg       0.69      0.64      0.65      4209



## Decision Tree + TF-IDF

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(XTrain, YTrain)
tree_pred = dt.predict(XTest)
print(tree_pred)

[False False  True ... False False  True]


In [ ]:
print(classification_report(YTest, tree_pred))

              precision    recall  f1-score   support

       False       0.45      0.65      0.53      1478
        True       0.75      0.57      0.65      2731

    accuracy                           0.60      4209
   macro avg       0.60      0.61      0.59      4209
weighted avg       0.64      0.60      0.60      4209



## Random Forest + TF-IDF

In [ ]:
train_data_features=XTrain.toarray()
forest = RandomForestClassifier(n_estimators = 150) 
forest = forest.fit( train_data_features, YTrain)

In [ ]:
test_data_features=XTest.toarray()
ypred = forest.predict(test_data_features)
print(ypred)

[False False False ... False False  True]


In [ ]:
print(classification_report(YTest, ypred))

              precision    recall  f1-score   support

       False       0.48      0.79      0.60      1478
        True       0.83      0.53      0.65      2731

    accuracy                           0.62      4209
   macro avg       0.65      0.66      0.62      4209
weighted avg       0.70      0.62      0.63      4209

